In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np



In [ ]:
results_url = "https://s3.amazonaws.com/dl.ncsbe.gov/ENRS/2025_11_04/results_pct_20251104.zip"

# Download the zip file into memory
response = requests.get(results_url)
response.raise_for_status()  # stop if there's an error

# Unzip directly in memory
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall("data/results_20251104")  # choose your destination folder
    print("Extracted files:", z.namelist())

df_og = pd.read_csv("data/results_20251104/results_pct_20251104.txt", sep="\t")

### MAKE SURE YOU REMOVE THIS ON ELECTION NIGHT ####
df_og['Total Votes'] = np.random.randint(0, 200, size=len(df_og))
### MAKE SURE YOU REMOVE THIS ON ELECTION NIGHT ####

df = df_og

#Make it so the Contest Name is not shouting
df['Contest Name'] = df['Contest Name'].str.title()
#Filter just for the transit tax
df = df[df['Contest Name']=='Mecklenburg County Public Transportation Sales And Use Tax']
#sum the total votes by precinct
df['total_precinct'] = df.groupby('Precinct')['Total Votes'].transform('sum')
#calcualte the percentage of votes per precinct for each choice
df['perc'] = ((df['Total Votes']/df['total_precinct'])*100).round(2)


#create a side-by-side for and against file
yes = df[df['Choice']=='For']
yes = yes[['Precinct', 'Total Votes', 'total_precinct', 'perc']]
yes.columns = ['prec_id', 'Votes For', 'total_precinct_votes', 'perc_yes']
no = df[df['Choice']=='Against']
no = no[['Precinct', 'Total Votes', 'perc']]
no.columns = ['prec_id', 'Votes Against', 'perc_no']

df = pd.merge(yes,no)

#exclude the rows that are not actually precincts
exclude = ['P-ABSENTEE BY MAIL', 'P-PROVISIONALS',  'P-TRANSFER']

df = df[~df['prec_id'].isin(exclude)]

#formatting for map viz
df['Percent For'] = df['perc_yes'].astype(str)
df['Percent For'] = df['Percent For'] + '%'

#calc the margin of percent points
df['Percentage Point Margin'] = df['perc_yes']-df['perc_no']
#calc the raw margin
df['Vote Margin'] = df['Votes For']-df['Votes Against']
#export 
df.to_excel('/content/drive/MyDrive/elections-2025/precinct-map/transit-results.xlsx', index=False)

In [7]:
# Read the shapefile
shapes = gpd.read_file("zip:////Users/cmcglade/Documents/Projects/elections/SBE_PRECINCTS_20250728.zip")

# Filter to Mecklenburg
shapes = shapes[shapes['county_nam']=='MECKLENBURG']

# Fix precinct IDs (optional)
shapes['prec_id'] = 'P-' + shapes['prec_id'].astype(str)

# Keep only columns needed for Flourish
shapes = shapes[['prec_id', 'geometry']]  # Remove other columns if needed

# Make sure it's a GeoDataFrame in WGS84
shapes = gpd.GeoDataFrame(shapes, geometry='geometry')
shapes = shapes.to_crs(epsg=4326)

shapes['geometry'] = shapes['geometry'].simplify(0.001)  # reduces vertex count


# Export clean GeoJSON
output_path = '/Users/cmcglade/Documents/Projects/elections/meck-precincts.geojson'
shapes.to_file(output_path, driver='GeoJSON')